Mapping Ocean Parcels output to conserved particles in Salish Sea Atlantis Boxes. 

In [1]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
from netCDF4 import Dataset
from shapely.geometry import Point

In [23]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"
data_df = gpd.read_file(shapefile_name)
data_df = data_df.sort_values(by=['BOX_ID'])
box_depth = data_df['BOTZ']
box_area = data_df['AREA']
box_volume = box_area * box_depth

#print(box_volume)

In [24]:
# Oil type properties & spill location selection

bitumen = {
    "Density": 1011.2, #kg/m^3
    "Naphthalene": 24, #mg/kg oil
    "Phenanthrene": 17,
    "Pyrene": 10,
    "Benzo": 3,
}

BunkerC = {
    "Density": 995.3,
    "Naphthalene": 680,
    "Phenanthrene": 796,
    "Pyrene": 266,
    "Benzo": 56,
}

Diesel = {
    "Density": 831.0,
    "Naphthalene": 3664,
    "Phenanthrene": 1000,
    "Pyrene": 0.000,
    "Benzo": 0.000,
}

Crude = {
    "Density": 884.7,
    "Naphthalene": 654,
    "Phenanthrene": 327,
    "Pyrene": 13,
    "Benzo": 2,
}

fuel_type = {
    "bitumen" : bitumen,
    "BunkerC" : BunkerC,
    "Diesel" : Diesel,
    "Crude" : Crude,
}

spill_volume = {
    "5b_Turn_Point_Diluted_bitumen" : 2000, #m^3
    "6a_VancouverHarbour_BunkerC" : 15,
    "7a_JohnsonStrait_BunkerC" : 1000,
    "4a_ActivePass_Diesel" : 500,
}

file_id = int(input( ))
scenario = {1 : "5b_Turn_Point_Diluted_bitumen",
            2 : "6a_VancouverHarbour_BunkerC",
            3 : "7a_JohnsonStrait_BunkerC",
            4 : "4a_ActivePass_Diesel",}

print("\nScenario running  :", scenario[file_id], sep = " ")


Scenario running  : 5b_Turn_Point_Diluted_bitumen


In [25]:
# Spill release times
release_start = '2018-01-01'  ## winter starts in December, Summer in Jul - Aug
release_end = '2018-01-02'
release_YYYYMM = '2018-01'
oil_per_particle = (fuel_type[scenario[file_id].split(sep = '_')[-1]]["Density"] * spill_volume[scenario[file_id]]) / 100
release_start_time = np.datetime64(release_start)
inputFileName = scenario[file_id] + str(release_start_time) + '_OP.nc'
#inputFileName = '4a_ActivePass_Diesel2020-07-01_OP.nc'

inputFileName


'5b_Turn_Point_Diluted_bitumen2018-01-01_OP.nc'

In [26]:
numLayers = 7
numSites = data_df.shape[0]
numTargetSites = numSites

#outputDT = 60*60
outputDT = 43100.00

stepsPerDay = int(86400.0/ outputDT)
numStepsPerDT = int(outputDT/3600.0)

debug = False

In [27]:
pfile = xr.open_dataset('results/'+str(inputFileName), decode_cf=True)

lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

In [28]:
numParticles = lon.shape[0]
trackDates = []

for i in range(0,numParticles):
    #print(time[i][0])
    trackDates.append(time[i][0])

RDiff = max(trackDates) - min(trackDates)
minDate = np.datetime64(release_start+"T00:30:00")
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

2018-01-01 00:30:00


In [29]:
numReleaseDays = 1
numReleaseSteps = numReleaseDays * stepsPerDay

trackLength = len(lon[0])

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))
numSteps = int(trackLength / numStepsPerDT)


trackLength = 145
numStepsPerDT = 11


In [30]:
# Create the netcdf output file

netcdfFileName = "BoxParticles_" + scenario[file_id] + "_" + str(release_start_time) + ".nc"
try:
    os.remove(netcdfFileName)
except:
    pass
ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)
Dataset.set_fill_on(ncfile)

# Dimensions
time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

In [31]:
# Variables
times = ncfile.createVariable("t",np.float64, ("t",))
oil = ncfile.createVariable("oil",np.float64,("t", "b"))
Naphthalene = ncfile.createVariable("Naphthalene",np.float64, ("t", "b", "z"))
Phenanthrene = ncfile.createVariable("Phenanthrene",np.float64,("t", "b", "z"))
Pyrene = ncfile.createVariable("Pyrene",np.float64,("t", "b", "z"))
Benzo = ncfile.createVariable("Benzo",np.float64,("t", "b", "z"))

# Attributes
Naphthalene.units = "Probability"
Naphthalene.long_name = "Naphthalene"
Naphthalene.missing_value = 0.0000
Naphthalene.valid_min = 0.0000
Naphthalene.valid_max = 100000000.0

Phenanthrene.units = "Probability"
Phenanthrene.long_name = "Phenanthrene"
Phenanthrene.missing_value = 0.0000
Phenanthrene.valid_min = 0.0000
Phenanthrene.valid_max = 100000000.0

Pyrene.units = "Probability"
Pyrene.long_name = "Pyrene"
Pyrene.missing_value = 0.0000
Pyrene.valid_min = 0.0000
Pyrene.valid_max = 100000000.0

Benzo.units = "Probability"
Benzo.long_name = "Benzo(a)pyrene"
Benzo.missing_value = 0.0000
Benzo.valid_min = 0.0000
Benzo.valid_max = 100000000.0

oil.units = "Probability"
oil.long_name = "Oil"

times.units = "seconds since " + d
times.dt = outputDT
times.long_name = "time"

In [32]:
# Populate variables with particle probabilities

timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData

Surface_particles = np.zeros((numSteps + numReleaseSteps, numTargetSites))
Dispersed_particles = np.zeros((numSteps + numReleaseSteps, numTargetSites, numLayers))

In [33]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)))

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT]
        partLat = lat[partIndex][stepIndex * numStepsPerDT]
        partProb = probs[partIndex][stepIndex * numStepsPerDT]

        matchFound = 0

        for targetIndex in range (0, numTargetSites):

            box_id = data_df.iloc[targetIndex].BOX_ID
            box_coordinates = data_df.iloc[targetIndex].geometry
            find_particle = box_coordinates.contains(Point(partLon, partLat))
            
            if box_depth[box_id] < 26:
                layer = 0
            elif box_depth[box_id] == 50:
                layer = 1
            elif box_depth[box_id] == 100:
                layer = 2
            elif box_depth[box_id] == 200:
                layer = 3
            elif box_depth[box_id] > 200 and box_depth[box_id] < 401:
                layer = 4
            elif box_depth[box_id] > 400:
                layer = 5

            if find_particle:
                Dispersed_particles[timeValue][box_id][layer] = Dispersed_particles[timeValue][box_id][layer] + partProb
                Surface_particles[timeValue][box_id] = Surface_particles[timeValue][box_id] + partProb
                
                break

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)

        #break

oil[:,:] = Surface_particles
Naphthalene[:,:,:] = Dispersed_particles
Phenanthrene[:,:,:] = Dispersed_particles
Pyrene[:,:,:] = Dispersed_particles
Benzo[:,:,:] = Dispersed_particles

ncfile.close()


In [34]:
np.histogram(Surface_particles)

(array([1913,   11,    5,    4,    9,    3,    4,    0,    0,    1]),
 array([ 0. ,  7.9, 15.8, 23.7, 31.6, 39.5, 47.4, 55.3, 63.2, 71.1, 79. ]))